In [2]:
# Year search trends data compiler. This allows you to access google search trends data and then:
# export it to a database, export to csv, or export as json with vertices and edges 
# This is different from the other file because it has a loop for each month of the year. 

# This file is the only one that works properly (kind of) with inputting to the database!!!!!
##########################################################
import csv
import pandas as pd
import json
from json import JSONEncoder
from pandas import DataFrame
from pytrends.request import TrendReq
import numpy as np

# Login to Google. Only need to run this once, the rest of requests will use the same session.
pytrend = TrendReq(hl='en-US', tz=360)

ModuleNotFoundError: No module named 'pytrends'

In [10]:
# List of search terms
keyword = (['symptomatic'])
location = ('US-UT')
timeframe = ('2020-01-01 2020-01-31',
             '2020-02-01 2020-02-29',
             '2020-03-01 2020-03-31',
             '2020-04-01 2020-04-30', 
             '2020-05-01 2020-05-31', 
             '2020-06-01 2020-06-30',
             '2020-07-01 2020-07-31',
             '2020-08-01 2020-08-31',
             '2020-09-01 2020-09-30', 
             '2020-10-01 2020-10-31',
             '2020-11-01 2020-11-30',
             '2020-12-01 2020-12-31')

filename = ('outputdata.txt')

In [11]:
print(keyword)

['symptomatic']


In [8]:
# Define an empty dataframe, and then add to it with each iteration (i.e. each month of data)

df = pd.DataFrame()
for time in timeframe: 
    # Create payload and capture API tokens 
    pytrend.build_payload(keyword, cat=0, timeframe=time,geo=location,gprop='')
    # Create dataframe of interest over time
    df2 = pytrend.interest_over_time()
    # Add new dataframe to df 
    df = df.append(df2)


#Format dataframe:
df = df.reset_index()
new_date_col = df['date'].dt.dayofyear
df.insert(loc=0, column='Date', value=new_date_col)
df.insert(loc=1, column='Location', value=location)
df = df.drop(columns = ['date', 'isPartial'])
keyword_str = ''.join(keyword)
df.columns=['Date','Location',f'Search: {keyword_str}']
df

,Date,Location,Search: symptomatic
0,61,US-UT,0
1,62,US-UT,0
2,63,US-UT,0
3,64,US-UT,0
4,65,US-UT,0
...,...,...,...
240,331,US-UT,0
241,332,US-UT,0
242,333,US-UT,0
243,334,US-UT,0


In [12]:
#EXPORTING TO DATABASE:
##########################################################
import sqlite3
from pandas import DataFrame

conn = sqlite3.connect('DATABASE.db')
c = conn.cursor()
conn.commit()

try:
    #this will fail if there is a new column
    df.to_sql("Search_Data",con=conn,if_exists='append',index=False)
except:
    data = pd.read_sql('SELECT * FROM Search_Data', conn)
    
    df2 = data.merge(df, on=['Date','Location'], how='left')
    df2 = df2.fillna(0)
    df2.to_sql("Search_Data", con=conn, if_exists = 'replace', index=False)

# conn.close()

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/generic.py:2653: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [38]:
# EXPORTING TO CSV FILE
##########################################################
df.to_csv('year_search_data.csv')

In [45]:
# EXPORT AS JSON WITH VERTICES/EDGES
##########################################################
# Create vertices: 
df_list = df.values.tolist()
vertices = { i : df_list[i] for i in range(0, len(df_list) ) }

# And edges:
edges = []
for i in range(1,len(df)):
    edge = [str(i-1),str(i)]
    edge = edges.append(edge)

finaldict = {"Data": { "Vertices": vertices, "Edges": edges}}
with open(filename, 'w') as outfile:
    json.dump(finaldict, outfile)